In [1]:
# pip install openai langchain neo4j speechrecognition pandas
# pip install ipykernel
# pip install langchain-openai
# pip install --upgrade langchain langchain-community langchain-openai
# pip install python-dotenv
# pip install python-docx
# pip install faiss-cpu openai
# pip install flask
# pip install python-docx
# pip install pyaudio
# pip install pyttsx3
# 
# 
# 

In [2]:
# import openai
# import neo4j
# import langchain
# import speech_recognition as sr
# import pandas as pd
# from neo4j import GraphDatabase
# from langchain.llms import OpenAI
# from langchain.chains import RetrievalQA
# from langchain.vectorstores import Neo4jVector

In [3]:
import os
import openai
import numpy as np
import speech_recognition as sr
import pyttsx3
from docx import Document
from neo4j import GraphDatabase, exceptions
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [5]:
# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

In [6]:
# Initialize text-to-speech engine
tts_engine = pyttsx3.init()

In [7]:
def speak_text(text):
    """Converts text to speech."""
    tts_engine.say(text)
    tts_engine.runAndWait()

In [8]:
def log_conversation(question, policy_name, answer):
    """Logs the conversation history."""
    with open("conversation_log.txt", "a") as file:
        file.write("\n==============================\n")
        file.write(f"**User Query:** {question}\n")
        file.write(f"**Relevant Policy:** {policy_name if policy_name else 'No policy found'}\n")
        file.write(f"**AI Response:** {answer if answer else 'No response generated'}\n")
        file.write("==============================\n")


In [9]:
def get_speech_input():
    """Captures speech input from the user."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for a policy-related question")
        try:
            audio = recognizer.listen(source)
            query = recognizer.recognize_google(audio)
            print(f"User Query: {query}")
            return query
        except sr.UnknownValueError:
            print("Could not understand the audio.")
            return None
        except sr.RequestError:
            print("Could not request results; check internet connection.")
            return None

In [10]:
def extract_text_from_docx(file_path):
    """Extracts text from a DOCX file."""
    try:
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

In [11]:
def load_policies_from_directory(directory):
    """Loads policy documents from the specified directory."""
    policies = {}
    if not os.path.exists(directory):
        print(f"Error: Policy directory '{directory}' not found.")
        return policies
    
    print(f"Scanning directory: {directory}")

    for file in os.listdir(directory):
        if file.endswith(".docx"):
            file_path = os.path.join(directory, file)
            content = extract_text_from_docx(file_path)
            if content.strip():
                policies[file] = content
                print(f"Loaded Policy: {file} - {len(content.split())} words")
            else:
                print(f"Skipping empty policy file: {file}")
    
    print(f"Total policies loaded: {len(policies)}")
    return policies

In [ ]:
# class PolicyDatabase:
#     """Handles policy storage and retrieval from Neo4j."""
#     def __init__(self, uri, user, password, policy_directory):
#         try:
#             self.driver = GraphDatabase.driver(uri, auth=(user, password))
#             with self.driver.session() as session:
#                 session.run("RETURN 1")
#             print("Connected to Neo4j successfully.")
#         except exceptions.ServiceUnavailable:
#             print("Error: Unable to connect to Neo4j. Ensure the database is running and credentials are correct.")
#             self.driver = None
#         except exceptions.AuthError:
#             print("Authentication Error: Invalid Neo4j username or password.")
#             self.driver = None
        
#         self.policies = load_policies_from_directory(policy_directory)
#         self.insert_policies_into_neo4j()
#         print("Policies successfully loaded into Neo4j.")

#     def insert_policies_into_neo4j(self):
#         """Inserts policies from the directory into Neo4j."""
#         if not self.policies:
#             print("No policies loaded from directory. Skipping Neo4j insertion.")
#             return
        
#         with self.driver.session() as session:
#             for policy_name, policy_content in self.policies.items():
#                 session.run(
#                     """
#                     MERGE (p:Policy {name: $name})
#                     SET p.content = $content
#                     """,
#                     parameters={"name": policy_name, "content": policy_content}
#                 )
#         print("Policies successfully inserted into Neo4j.")

#     def close(self):
#         """Closes the Neo4j database connection."""
#         if self.driver:
#             self.driver.close()
#             print("Neo4j connection closed.")

#     def find_relevant_policy_in_neo4j(self, question):
#         """Searches for the most relevant policy in Neo4j using full-text search."""
#         with self.driver.session() as session:
#             result = session.run(
#                 """
#                 CALL db.index.fulltext.queryNodes('policyIndex', $query) 
#                 YIELD node, score
#                 RETURN node.name AS name, node.content AS content
#                 ORDER BY score DESC
#                 LIMIT 1
#                 """,
#                 parameters={"query": question}
#             )
#             policy_data = result.single()

#         if policy_data:
#             print(f"Policy Found in Neo4j: {policy_data['name']}")
#         else:
#             print("No policy found in Neo4j for this query.")

#         return (policy_data["name"], policy_data["content"]) if policy_data else (None, None)

#     def ask_policy_ai(self, question):
#         """Retrieves the most relevant policy and generates an AI response."""

#         # Search in local policies first
#         for policy_name, policy_content in self.policies.items():
#             if question.lower() in policy_content.lower():
#                 print(f"Found relevant policy in directory: {policy_name}")
#                 relevant_policy_name = policy_name
#                 relevant_policy_content = policy_content
#                 break
#         else:
#             print("No relevant policy found in directory. Searching in Neo4j...")
#             relevant_policy_name, relevant_policy_content = self.find_relevant_policy_in_neo4j(question)

#         if not relevant_policy_content:
#             print("No matching policy found in Neo4j or directory.")
#             return None

#         print(f"Retrieving response based on policy: {relevant_policy_name}")
#         prompt = f"Based on the following policy '{relevant_policy_name}', answer: {question}\n{relevant_policy_content}"

#         try:
#             response = client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a policy expert. Answer user queries based on policy documents."},
#                     {"role": "user", "content": prompt}
#                 ]
#             )

#             answer = response.choices[0].message.content
#             print(f"AI Response: {answer}")
#             return answer

#         except Exception as e:
#             print(f"Error calling OpenAI API: {e}")
#             return None
    
# policy_directory = "policies"
# policy_db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, policy_directory)

# if policy_db.driver:
#     while True:
#         spoken_question = get_speech_input()
#         if spoken_question:
#             if "exit" in spoken_question.lower():
#                 print("Ending conversation.")
#                 speak_text("Goodbye. Have a great day.")
#                 break
#             policy_db.ask_policy_ai(spoken_question)
    
#     policy_db.close()
# else:
#     print("Database connection failed. Exiting.")

Connected to Neo4j successfully.
Scanning directory: policies
Loaded Policy: 6. Foreign Transaction Fees and Currency Conversion Policy.docx - 1000 words
Loaded Policy: 1. Cardholder Agreement Terms.docx - 814 words
Loaded Policy: 8. Promotional Offers and Balance Transfers Policy .docx - 992 words
Loaded Policy: 5. Dispute Resolution Policy.docx - 956 words
Loaded Policy: 4. Rewards Program Terms.docx - 846 words
Loaded Policy: 9. Privacy and Data Sharing Policy.docx - 903 words
Error reading policies/~$ Fraud Protection and Chargeback Policy.docx: Package not found at 'policies/~$ Fraud Protection and Chargeback Policy.docx'
Skipping empty policy file: ~$ Fraud Protection and Chargeback Policy.docx
Loaded Policy: 3. Fraud Protection and Chargeback Policy.docx - 910 words
Loaded Policy: 10. Legal and Compliance Policy.docx - 998 words
Loaded Policy: 2. Billing & Payment Policy.docx - 791 words
Loaded Policy: 7. Card Benefits (Travel Insurance, Purchase Protection, and Warranty).docx -

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such fulltext schema index: policyIndex}

In [13]:
what is the maximum time frame a cardholder has to report unauthorized or fraudulent transactions to remain protected?

Object `protected` not found.


In [14]:
what is the maximum time frame a cardholder has to report unauthorized or fraudulent transactions to remain protected?

Object `protected` not found.


In [15]:
# import os

# # Change to your correct project directory
# os.chdir("/Users/drashteeparmar/Drashtee/Programming/AI_Powered_Policy_Compliance_System/")

# # Verify the new working directory
# print("New Directory:", os.getcwd())

# print(".env Exists:", os.path.exists(".env"))

In [16]:
# import os
# from dotenv import load_dotenv, dotenv_values

# # Explicitly specify the .env file path
# dotenv_path = os.path.abspath(".env")

# # Load the .env file
# load_dotenv(dotenv_path=dotenv_path)

# # Debugging: Read .env without loading it into os.environ
# config = dotenv_values(dotenv_path)
# # print("Config Output:", config)  # Should print key-value pairs

# # Retrieve API Key
# api_key = os.getenv("OPENAI_API_KEY")
# print("OpenAI API Key:", "Loaded successfully!" if api_key else "Not found!")

Create and Insert Data in Neo4j















In [17]:
# 













In [18]:
# import os
# from docx import Document
# from neo4j import GraphDatabase

# class PolicyDatabase:
#     def __init__(self, uri, user, password):
#         """Initialize connection to Neo4j."""
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()
#         print("Neo4j connection closed.")

#     def initialize_schema(self):
#         print("Creating Neo4j schema...")
#         with self.driver.session() as session:
#             session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Policy) REQUIRE p.id IS UNIQUE;")
#             print("Schema created successfully!")

#     def insert_policies_from_directory(self, directory):
#         print(f"Searching for policies in '{directory}'...")

#         if not os.path.exists(directory):
#             print(f" Directory '{directory}' not found!")
#             return

#         policy_id = 1
#         with self.driver.session() as session:
#             for file in os.listdir(directory):
#                 if file.endswith(".docx"):
#                     file_path = os.path.join(directory, file)
#                     print(f" Processing: {file_path}")

#                     # Read document content
#                     doc = Document(file_path)
#                     content = "\n".join([para.text for para in doc.paragraphs])

#                     # Insert into Neo4j
#                     session.run("""
#                         MERGE (p:Policy {id: $id})
#                         SET p.name = $name, p.content = $content
#                     """, id=policy_id, name=file, content=content)
                    
#                     print(f" Inserted: {file}")
#                     policy_id += 1
    
#     def search_policies(self, keyword):
#         """Search for policies based on keywords."""
#         print(f"🔍 Searching for policies with keyword: '{keyword}'...")
#         with self.driver.session() as session:
#             result = session.run("""
#                 MATCH (p:Policy)
#                 WHERE p.content CONTAINS $keyword
#                 RETURN p.name, p.content LIMIT 5;
#             """, keyword=keyword)

#             policies = result.data()
#             if policies:
#                 for policy in policies:
#                     print(f" Found: {policy['p.name']}")
#                     print(f" Content: {policy['p.content'][:200]}...\n")
#             else:
#                 print(" No policies found!")

#     def ask_policy_ai(self, question):
#         """Use AI (OpenAI) to analyze policy documents and answer questions."""
#         print(f" AI is analyzing: '{question}'...")

#         # Retrieve all policy content
#         with self.driver.session() as session:
#             result = session.run("MATCH (p:Policy) RETURN p.content;")
#             policies = [record["p.content"] for record in result]

#         if not policies:
#             print(" No policies found!")
#             return

#         # Combine all policies into one prompt
#         context = "\n\n".join(policies)
#         prompt = f"Based on the following policies, answer: {question}\n\n{context}"

#         # Send query to OpenAI API
#         response = openai.ChatCompletion.create(
#             model="gpt-4",
#             messages=[{"role": "system", "content": "You are a policy expert."},
#                       {"role": "user", "content": prompt}]
#         )

#         answer = response["choices"][0]["message"]["content"]
#         print(" AI Response:", answer)


# # # Initialize and set up the database
# # db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# # db.initialize_schema()
# # db.insert_policies_from_directory("policies")  # Load all policy files
# # db.search_policies("refund")  # Example: Search for "refund" in policies
# # db.ask_policy_ai("What is the refund policy?")  # AI-based query
# # db.close()

# # Initialize and set up the database
# db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# db.initialize_schema()
# db.insert_policies_from_directory("policies")  # Load all policy files
# db.close()

Please check those policies on:

http://localhost:7474/browser/

Query: MATCH (p:Policy) RETURN p.id, p.name, LEFT(p.content, 100) ;

In [19]:
# from openai import OpenAI

# class PolicyDatabase:
#     def __init__(self, uri, user, password):
#         """Initialize connection to Neo4j."""
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))
#         self.client = OpenAI()  # Initialize OpenAI Client

#     def close(self):
#         """Close the Neo4j connection."""
#         self.driver.close()
#         print(" Neo4j connection closed.")

#     def ask_policy_ai(self, question):
#         """Use AI (OpenAI) to analyze policy documents and answer questions."""
#         print(f" AI is analyzing: '{question}'...")

#         # Retrieve all policy content
#         with self.driver.session() as session:
#             result = session.run("MATCH (p:Policy) RETURN p.content;")
#             policies = [record["p.content"] for record in result]

#         if not policies:
#             print(" No policies found!")
#             return None

#         # Combine all policies into one prompt
#         context = "\n\n".join(policies)
#         prompt = f"Based on the following policies, answer: {question}\n\n{context}"

#         print(" Sending request to OpenAI...")
#         print(f" Prompt (First 500 chars):\n{prompt[:500]}...")

#         try:
#             # New OpenAI API format
#             response = self.client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
#                     {"role": "user", "content": prompt}
#                 ]
#             )

#             print(" Received response from OpenAI API:")
#             print(response)  # Print full API response for debugging

#             # Extract answer
#             answer = response.choices[0].message.content
#             print(" AI Response:", answer)
#             return answer

#         except Exception as e:
#             print(" Error calling OpenAI API:", e)

In [20]:
# import speech_recognition as sr

# for index, name in enumerate(sr.Microphone.list_microphone_names()):
#     print(f" Microphone {index}: {name}")

In [2]:
import os
import openai
import numpy as np
import speech_recognition as sr
import pyttsx3
from docx import Document
from neo4j import GraphDatabase
from dotenv import load_dotenv
from openai import OpenAI

# Load API keys from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

# Initialize Text-to-Speech Engine
tts_engine = pyttsx3.init()

def speak_text(text):
    """Convert text to speech."""
    tts_engine.say(text)
    tts_engine.runAndWait()

def log_conversation(question, answer, policies_used):
    """Save the conversation in a text file."""
    with open("conversation_log.txt", "a") as file:
        file.write(f"Question: {question}\n")
        file.write(f"Policies Used: {policies_used}\n")
        file.write(f"Answer: {answer}\n")
        file.write("*" * 50 + "\n")

class PolicyDatabase:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Connected to Neo4j successfully.")

    def close(self):
        self.driver.close()
        print("Neo4j connection closed.")

    def extract_text_from_docx(self, file_path):
        try:
            doc = Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return ""

    def insert_policies_from_directory(self, directory):
        print(f"Searching for policies in '{directory}'...")
        if not os.path.exists(directory):
            print(f"Directory '{directory}' not found!")
            return
        
        policy_id = 1
        with self.driver.session() as session:
            for file in os.listdir(directory):
                if file.endswith(".docx"):
                    file_path = os.path.join(directory, file)
                    print(f"Processing: {file_path}")
                    content = self.extract_text_from_docx(file_path)
                    if not content.strip():
                        print(f"Skipping empty document: {file}")
                        continue
                    
                    session.run(
                        """
                        MERGE (p:Policy {id: $id})
                        SET p.name = $name, p.content = $content
                        """, id=policy_id, name=file, content=content)
                    print(f"Inserted: {file}")
                    policy_id += 1

    def find_relevant_policies(self, question):
        """Finds the most relevant policies using OpenAI embeddings."""
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.name, p.content;")
            policies = [(record["p.name"], record["p.content"]) for record in result]

        if not policies:
            print("No policies found in the database.")
            return []

        question_embedding = client.embeddings.create(
            model="text-embedding-ada-002",
            input=question
        ).data[0].embedding

        policy_embeddings = [
            (
                policy_name,
                policy_content,
                client.embeddings.create(model="text-embedding-ada-002", input=policy_content[:500]).data[0].embedding
            )
            for policy_name, policy_content in policies
        ]

        def cosine_similarity(vec1, vec2):
            return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

        ranked_policies = sorted(
            policy_embeddings,
            key=lambda x: cosine_similarity(question_embedding, x[2]),
            reverse=True
        )

        return [(p[0], p[1]) for p in ranked_policies[:3]]

    def ask_policy_ai(self, question):
        print(f"AI is analyzing: '{question}'...")
        relevant_policies = self.find_relevant_policies(question)

        if not relevant_policies:
            print("No directly matching policies found, sending all policies to AI.")
            with self.driver.session() as session:
                result = session.run("MATCH (p:Policy) RETURN p.name, p.content;")
                policies = [(record["p.name"], record["p.content"]) for record in result]
            context = "\n\n".join([f"{p[0]}: {p[1]}" for p in policies[:3]])
            extracted_policy_names = "All available policies"
        else:
            context = "\n\n".join([f"{p[0]}: {p[1]}" for p in relevant_policies])
            extracted_policy_names = ", ".join([p[0] for p in relevant_policies])

        print(f"Extracted Policies: {extracted_policy_names}")
        
        prompt = f"Based on the following policies: {extracted_policy_names}, answer: {question}\n\n{context}"
        
        print("Sending request to OpenAI...")
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a policy expert. Answer user queries based on policy documents."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            ai_answer = response.choices[0].message.content
            print("Final Result:")
            print(f"Extracted Policies: {extracted_policy_names}")
            print(f"AI Response: {ai_answer}")
            
            self.store_conversation_in_db(question, ai_answer, extracted_policy_names)
            return ai_answer
        except Exception as e:
            print("Error calling OpenAI API:", e)
            return None
    
    def store_conversation_in_db(self, question, answer, policies_used):
        with self.driver.session() as session:
            session.run(
                """
                CREATE (c:Conversation {question: $question, answer: $answer, policies: $policies})
                """, question=question, answer=answer, policies=policies_used)
        print("Conversation stored in Neo4j with extracted policies.")

# Speech-to-Text Function
def get_speech_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("*" * 50)
        print("Listening... Speak your policy-related question:")
        try:
            audio = recognizer.listen(source)
            query = recognizer.recognize_google(audio)
            print(f"You asked: {query}")
            return query
        except sr.UnknownValueError:
            print("Speech Recognition could not understand the audio")
            return None
        except sr.RequestError:
            print("Could not request results, check your internet connection")
            return None

policy_db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
policy_db.insert_policies_from_directory("policies")

while True:
    spoken_question = get_speech_input()
    if spoken_question:
        if "exit" in spoken_question.lower():
            print("Exiting conversation.")
            speak_text("Goodbye! Have a great day!")
            break
        policy_db.ask_policy_ai(spoken_question)

policy_db.close()

Connected to Neo4j successfully.
Searching for policies in 'policies'...
Processing: policies/6. Foreign Transaction Fees and Currency Conversion Policy.docx
Inserted: 6. Foreign Transaction Fees and Currency Conversion Policy.docx
Processing: policies/1. Cardholder Agreement Terms.docx
Inserted: 1. Cardholder Agreement Terms.docx
Processing: policies/8. Promotional Offers and Balance Transfers Policy .docx
Inserted: 8. Promotional Offers and Balance Transfers Policy .docx
Processing: policies/5. Dispute Resolution Policy.docx
Inserted: 5. Dispute Resolution Policy.docx
Processing: policies/4. Rewards Program Terms.docx
Inserted: 4. Rewards Program Terms.docx
Processing: policies/9. Privacy and Data Sharing Policy.docx
Inserted: 9. Privacy and Data Sharing Policy.docx
Processing: policies/~$ Fraud Protection and Chargeback Policy.docx
Error reading policies/~$ Fraud Protection and Chargeback Policy.docx: Package not found at 'policies/~$ Fraud Protection and Chargeback Policy.docx'
Ski

In [ ]:
# import os
# import openai
# import numpy as np
# import speech_recognition as sr
# import pyttsx3
# from docx import Document
# from neo4j import GraphDatabase, exceptions
# from dotenv import load_dotenv
# from openai import OpenAI

# # Load environment variables
# load_dotenv()
# openai_api_key = os.getenv("OPENAI_API_KEY")
# NEO4J_URI = os.getenv("NEO4J_URI")
# NEO4J_USER = os.getenv("NEO4J_USER")
# NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# # Initialize OpenAI client
# client = OpenAI(api_key=openai_api_key)

# # Initialize text-to-speech engine
# tts_engine = pyttsx3.init()

# def speak_text(text):
#     """Converts text to speech."""
#     tts_engine.say(text)
#     tts_engine.runAndWait()

# def log_conversation(question, policy_name, answer):
#     """Logs the conversation history."""
#     with open("conversation_log.txt", "a") as file:
#         file.write("\n==============================\n")
#         file.write(f"User Query: {question}\n")
#         file.write(f"Relevant Policy: {policy_name if policy_name else 'No policy found'}\n")
#         file.write(f"AI Response: {answer if answer else 'No response generated'}\n")
#         file.write("==============================\n")

# def get_speech_input():
#     """Captures speech input from the user."""
#     recognizer = sr.Recognizer()
#     with sr.Microphone() as source:
#         print("Listening for a policy-related question")
#         try:
#             audio = recognizer.listen(source)
#             query = recognizer.recognize_google(audio)
#             print(f"User Query: {query}")
#             return query
#         except sr.UnknownValueError:
#             print("Could not understand the audio.")
#             return None
#         except sr.RequestError:
#             print("Could not request results; check internet connection.")
#             return None

# def extract_text_from_docx(file_path):
#     """Extracts text from a DOCX file."""
#     try:
#         doc = Document(file_path)
#         return "\n".join([para.text for para in doc.paragraphs])
#     except Exception as e:
#         print(f"Error reading {file_path}: {e}")
#         return ""

# def load_policies_from_directory(directory):
#     """Loads policy documents from a specified directory."""
#     policies = {}
#     if not os.path.exists(directory):
#         print(f"Policy directory '{directory}' not found.")
#         return policies
    
#     for file in os.listdir(directory):
#         if file.endswith(".docx"):
#             file_path = os.path.join(directory, file)
#             content = extract_text_from_docx(file_path)
#             if content.strip():
#                 policies[file] = content
#                 print(f"Loaded Policy: {file}")
#             else:
#                 print(f"Skipping empty policy file: {file}")
#     return policies

# class PolicyDatabase:
#     """Handles policy storage and retrieval from Neo4j."""
#     def __init__(self, uri, user, password, policy_directory):
#         try:
#             self.driver = GraphDatabase.driver(uri, auth=(user, password))
#             with self.driver.session() as session:
#                 session.run("RETURN 1")
#             print("Connected to Neo4j successfully.")
#         except exceptions.ServiceUnavailable:
#             print("Error: Unable to connect to Neo4j. Ensure the database is running and credentials are correct.")
#             self.driver = None
#         except exceptions.AuthError:
#             print("Authentication Error: Invalid Neo4j username or password.")
#             self.driver = None
        
#         self.policies = load_policies_from_directory(policy_directory)
#         self.insert_policies_into_neo4j()
#         print("Policies successfully loaded into Neo4j.")

#     def insert_policies_into_neo4j(self):
#         """Inserts policy documents into Neo4j for optimized searching."""
#         with self.driver.session() as session:
#             for policy_name, policy_content in self.policies.items():
#                 session.run(
#                     """
#                     MERGE (p:Policy {name: $name})
#                     SET p.content = $content
#                     """,
#                     parameters={"name": policy_name, "content": policy_content}
#                 )

#     def close(self):
#         """Closes the Neo4j database connection."""
#         if self.driver:
#             self.driver.close()
#             print("Neo4j connection closed.")

#     def find_relevant_policy_in_neo4j(self, question):
#         """Searches for the most relevant policy in Neo4j based on user query."""
#         with self.driver.session() as session:
#             result = session.run(
#                 """
#                 MATCH (p:Policy)
#                 WHERE p.content CONTAINS $query
#                 RETURN p.name, p.content
#                 LIMIT 1
#                 """,
#                 parameters={"query": question}
#             )
#             policy_data = result.single()
#         return (policy_data["p.name"], policy_data["p.content"]) if policy_data else (None, None)

#     def ask_policy_ai(self, question):
#         """Retrieves the most relevant policy and generates an AI response."""
#         if not self.driver:
#             print("Database connection is not available.")
#             return None

#         print("Searching Neo4j for relevant policy")
#         policy_name, policy_content = self.find_relevant_policy_in_neo4j(question)
        
#         if not policy_content:
#             print("No matching policy found in the database.")
#             return None
        
#         print(f"Relevant Policy Found: {policy_name}")
        
#         prompt = f"Based on the following policy '{policy_name}', answer: {question}\n{policy_content}"
        
#         print("Querying OpenAI for a response")
#         try:
#             response = client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a policy expert. Answer user queries based on policy documents."},
#                     {"role": "user", "content": prompt}
#                 ]
#             )
            
#             answer = response.choices[0].message.content
#             print("==============================")
#             print(f"User Query: {question}")
#             print(f"Relevant Policy: {policy_name}")
#             print(f"AI Response: {answer}")
#             print("==============================")
#             speak_text(answer)
#             log_conversation(question, policy_name, answer)
#             return answer
#         except Exception as e:
#             print("Error calling OpenAI API:", e)
#             return None

# policy_directory = "policies"
# policy_db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, policy_directory)

# if policy_db.driver:
#     while True:
#         spoken_question = get_speech_input()
#         if spoken_question:
#             if "exit" in spoken_question.lower():
#                 print("Ending conversation.")
#                 speak_text("Goodbye. Have a great day.")
#                 break
#             policy_db.ask_policy_ai(spoken_question)
    
#     policy_db.close()
# else:
#     print("Database connection failed. Exiting.")


Connected to Neo4j successfully.
Loaded Policy: 6. Foreign Transaction Fees and Currency Conversion Policy.docx
Loaded Policy: 1. Cardholder Agreement Terms.docx
Loaded Policy: 8. Promotional Offers and Balance Transfers Policy .docx
Loaded Policy: 5. Dispute Resolution Policy.docx
Loaded Policy: 4. Rewards Program Terms.docx
Loaded Policy: 9. Privacy and Data Sharing Policy.docx
Error reading policies/~$ Fraud Protection and Chargeback Policy.docx: Package not found at 'policies/~$ Fraud Protection and Chargeback Policy.docx'
Skipping empty policy file: ~$ Fraud Protection and Chargeback Policy.docx
Loaded Policy: 3. Fraud Protection and Chargeback Policy.docx
Loaded Policy: 10. Legal and Compliance Policy.docx
Loaded Policy: 2. Billing & Payment Policy.docx
Loaded Policy: 7. Card Benefits (Travel Insurance, Purchase Protection, and Warranty).docx
Policies successfully loaded into Neo4j.
Listening for a policy-related question
User Query: how can card holder freeze their card
Searchin